Chat GPT link:
https://chat.openai.com/share/94908c64-c5d8-4eb5-be83-77dc3c5ee8cb

# Day 5: If You Give A Seed A Fertilizer

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```


The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

```
50 98 2
52 50 48
```


The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```


With this map, you can look up the soil number required for each initial seed number:

    Seed number 79 corresponds to soil number 81.
    Seed number 14 corresponds to soil number 14.
    Seed number 55 corresponds to soil number 57.
    Seed number 13 corresponds to soil number 13.

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

    Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
    Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
    Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
    Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [1]:
from typing import List, Dict, Tuple

def parse_input_file(file_path: str) -> Tuple[List[int], List[Dict[str, List[Dict[str, int]]]]]:
    """
    Parse the input file and extract relevant information.

    Args:
    - file_path (str): The path to the input file.

    Returns:
    - Tuple[List[int], List[Dict[str, List[Dict[str, int]]]]]: A tuple containing:
      1. List of seed numbers.
      2. List of dictionaries for each map, with information about the map.

    Example:
    >>> parse_input_file("input.txt")
    ([10, 20, 50], [{'seed_to_soil_map': [{'destination_range_start': 50, 'source_range_start': 98, 'range_length': 2}, ...]},
                   {'soil_to_fertilizer_map': [{'destination_range_start': 0, 'source_range_start': 15, 'range_length': 37}, ...]}])
    """
    seed_numbers = []
    maps_data = []

    with open(file_path, 'r') as file:
        lines = file.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        i += 1

        if line.startswith("seeds:"):
            seed_numbers = list(map(int, line.split()[1:]))
        elif line.endswith("map:"):
            map_name = line.split()[0].lower()
            map_data = []

            while i < len(lines) and lines[i].strip():
                map_row = list(map(int, lines[i].split()))
                map_data.append({
                    'destination_range_start': map_row[0],
                    'source_range_start': map_row[1],
                    'range_length': map_row[2],
                })
                i += 1

            maps_data.append({f'{map_name}_map': map_data})

    return seed_numbers, maps_data


In [ ]:
file_path = "Chris_Davis_input_day5.txt"
#file_path = 'test_input.txt'
seeds, maps = parse_input_file(file_path)
print("Seed Numbers:", seeds)
print("Maps Data:", maps)

In [ ]:
from typing import List, Dict, Tuple

def process_maps(maps_data: List[Dict[str, List[Dict[str, int]]]]) -> List[Dict[str, List[Dict[str, int]]]]:
    """
    Add 'source_type' and 'destination_type' keys to the outer dictionary for each map.

    Args:
    - maps_data (List[Dict[str, List[Dict[str, int]]]]): List of dictionaries representing maps.

    Returns:
    - List[Dict[str, List[Dict[str, int]]]]: Updated list of dictionaries with 'source_type' and 'destination_type' keys.

    Example:
    >>> process_maps([{'seed_to_soil_map': [{'destination_range_start': 50, 'source_range_start': 98, 'range_length': 2}, ...]}])
    [{'source_type': 'seed', 'destination_type': 'soil', 'map': [{'destination_range_start': 50, 'source_range_start': 98, 'range_length': 2}, ...]}]
    """
    processed_maps = []

    for map_entry in maps_data:
        processed_map_entry = {}
        for map_name, map_list in map_entry.items():
            source_type, destination_type = map_name.split('-to-')
            
            processed_list = []
            for entry in map_list:
                entry['source_type'] = source_type
                entry['destination_type'] = destination_type.strip('_map')
                processed_list.append(entry)

            # Add the keys to the outer dictionary
            processed_map_entry['source_type'] = source_type
            processed_map_entry['destination_type'] = destination_type.strip('_map')
            processed_map_entry['map'] = processed_list

        processed_maps.append(processed_map_entry)

    return processed_maps

In [4]:
processed_maps = process_maps(maps)

In [5]:
import logging
from typing import List, Dict, Union

# Configure logging
logging.basicConfig(level=logging.INFO)

def traverse_maps(seed: int, source_type: str, destination_type: str, maps: List[Dict[str, List[Dict[str, Union[int, str]]]]]) -> Union[int, None]:
    """
    Traverse through the maps based on source_type and destination_type.

    Args:
    - seed (int): The initial seed value.
    - source_type (str): The initial source type.
    - destination_type (str): The final destination type.
    - maps (List[Dict[str, List[Dict[str, Union[int, str]]]]]): List of dictionaries representing maps.

    Returns:
    - Union[int, None]: The final value or None if the destination type is not found.

    Example:
    >>> seed_to_soil_map = [{'destination_range_start': 50, 'source_range_start': 98, 'range_length': 2, 'source_type': 'seed', 'destination_type': 'soil'}]
    >>> maps_data = [{'source_type': 'seed', 'destination_type': 'soil', 'map': seed_to_soil_map}]
    >>> traverse_maps(10, 'seed', 'location', maps_data)
    52
    """
    current_value = seed

    # Iterate until the final destination type is reached
    while source_type != destination_type:
        # Find the map corresponding to the current source type
        current_map = next((map_data for map_data in maps if map_data['source_type'] == source_type), None)

        if current_map is None:
            # No matching map found
            logging.info(f"No matching map found for source type '{source_type=}'")
            return None

        # Find the entry corresponding to the current value in the map
        map_entry = next((entry for entry in current_map['map'] if entry['source_range_start'] <= current_value <= (entry['source_range_start'] + entry['range_length'])), None)

        
        if map_entry is None:
            # No matching entry found, set destination value to be the same as source value
            logging.info(f"No matching entry found for source value '{current_value=}' in map for source type '{source_type=}'. Setting destination value to be the same.")
            current_value = current_value
            source_type = destination_type  # Skip to the next type
        
        else:    
            print(f'{map_entry=}\n')
            print(f'{current_value=}\n')
            delta = current_value - map_entry['source_range_start']
            print(f'{delta=}\n')
            # Log the matching values
            logging.info(f"Matching values: source_type='{source_type}', current_value='{current_value}', destination_type='{map_entry['destination_type']}', new_value='{map_entry['destination_range_start']} + {delta=}'")

            # Update the current value and source type for the next iteration
            current_value = map_entry['destination_range_start'] + delta
            source_type = map_entry['destination_type']

    return current_value

In [6]:
import logging
from typing import List, Dict, Union

# Configure logging
logging.basicConfig(level=logging.INFO)

def traverse_maps(seed: int, source_type: str, destination_type: str, maps: List[Dict[str, List[Dict[str, Union[int, str]]]]]) -> Union[int, None]:
    """
    Traverse through the maps based on source_type and destination_type, moving to the next destination if no match is found.

    Args:
    - seed (int): The initial seed value.
    - source_type (str): The initial source type.
    - destination_type (str): The final destination type.
    - maps (List[Dict[str, List[Dict[str, Union[int, str]]]]]): List of dictionaries representing maps.

    Returns:
    - Union[int, None]: The final value or None if the destination type is not found.

    Example:
    >>> seed_to_soil_map = [{'destination_range_start': 50, 'source_range_start': 98, 'range_length': 2, 'source_type': 'seed', 'destination_type': 'soil'}]
    >>> maps_data = [{'source_type': 'seed', 'destination_type': 'soil', 'map': seed_to_soil_map}]
    >>> traverse_maps(10, 'seed', 'location', maps_data)
    52
    """
    current_value = seed

    # Iterate until the final destination type is reached
    while source_type != destination_type:
        # Find the map corresponding to the current source type
        current_map = next((map_data for map_data in maps if map_data['source_type'] == source_type), None)

        if current_map is None:
            # No matching map found
            logging.info(f"No matching map found for source type {source_type=}")
            return None

        # Find the entry corresponding to the current value in the map
        map_entry = next((entry for entry in current_map['map'] if entry['source_range_start'] <= current_value <= (entry['source_range_start'] + entry['range_length'])), None)

        if map_entry is None:
            # No matching entry found, move to the next destination in the map
            logging.info(f"No matching entry found for source value {current_value=} in map for source type {source_type=}. Moving to the next destination.")
            
            # Find the next destination in the map
            next_destination_entry = next((map_data for map_data in maps if map_data['source_type'] == current_map['destination_type']), None)

            if next_destination_entry is None and current_map['destination_type'] == destination_type:
                # No next destination found, return None
                logging.info(f"No next destination found in map for source type {current_map['destination_type']=}.")
                source_type = destination_type

            # Update the source type for the next iteration
            else:
                source_type = next_destination_entry['source_type']
            logging.info(f'Moving on to source type {source_type}')
                                    
        else:
            # Update the current value and source type for the next iteration
            delta = current_value - map_entry['source_range_start']
            
            
            # Logging before updating matching values
            logging.info(f"Matching values: {source_type=}, {current_value=}, destination_type={map_entry['destination_type']}, new_value={map_entry['destination_range_start']} + {delta=} = {map_entry['destination_range_start'] + delta}")
            
            current_value = map_entry['destination_range_start'] + delta
            source_type = map_entry['destination_type']

            
    return current_value

In [7]:
def get_seed_locations(seeds: List[int], processed_maps: List[Dict[str, List[Dict[str, Union[int, str]]]]], source_type: str = 'seed', destination_type: str = 'location') -> Dict[int, Union[int, None]]:
    """
    Get the locations for each seed using the traverse_maps_with_defaults function.

    Args:
    - seeds (List[int]): List of seed values.
    - processed_maps (List[Dict[str, List[Dict[str, Union[int, str]]]]]): List of dictionaries representing processed maps.
    - source_type (str, optional): The initial source type. Defaults to 'seed'.
    - destination_type (str, optional): The final destination type. Defaults to 'location'.

    Returns:
    - Dict[int, Union[int, None]]: A dictionary where keys are seed values and values are corresponding locations.
    """
    seed_locations = {}

    for seed in seeds:
        location = traverse_maps(seed, source_type, destination_type, processed_maps)
        seed_locations[seed] = location

    return seed_locations

In [8]:
seed_locations = get_seed_locations(seeds, processed_maps)

INFO:root:Matching values: source_type='seed', current_value=2276375722, destination_type=soil, new_value=2745421185 + delta=38100697 = 2783521882
INFO:root:Matching values: source_type='soil', current_value=2783521882, destination_type=fertilizer, new_value=1129998067 + delta=538597497 = 1668595564
INFO:root:Matching values: source_type='fertilizer', current_value=1668595564, destination_type=water, new_value=1339983969 + delta=134198805 = 1474182774
INFO:root:Matching values: source_type='water', current_value=1474182774, destination_type=light, new_value=500839409 + delta=7700992 = 508540401
INFO:root:No matching entry found for source value current_value=508540401 in map for source type source_type='light'. Moving to the next destination.
INFO:root:Moving on to source type temperature
INFO:root:Matching values: source_type='temperature', current_value=508540401, destination_type=humidity, new_value=3733360236 + delta=64167573 = 3797527809
INFO:root:Matching values: source_type='hum

In [ ]:
seed_locations

In [10]:
min_value = min(seed_locations.values())

In [11]:
min_value

484023871

## Summary of part 1 from ChatGPT:

prompt: `Great! We can now move on to the next part. Before we do so, can you summarize what we did and what you think the problem we had to solve was?`

Certainly! Here's a summary of what we've done so far and the problem we've been addressing:
Summary:

    File Parsing Function:
        Created a function parse_input_file that reads an input file containing information about seeds and various maps.
        Extracted seed numbers and organized map data into a list of dictionaries.

    Map Processing Function:
        Implemented a function process_maps to add 'source_type' and 'destination_type' keys to the outer dictionary for each map.
        Modified the function to add these keys to the outer dictionary instead of the inner.

    Map Traversal Function:
        Developed a function traverse_maps to navigate through maps based on source and destination types.
        Added logging statements for debugging and tracking matching values during traversal.

    Finding Minimum Value in Dictionary:
        Provided code snippets to find the key corresponding to the minimum value in a dictionary.

Problem Addressed:

The problem we've been addressing involves processing an input file containing seed information and maps. The goal is to extract seed numbers and organize map data, then navigate through the maps based on source and destination types. Additionally, we needed to log information for debugging purposes and find the minimum value in a dictionary.

Please let me know if there are specific aspects you'd like more details on or if you have any additional questions!

# Part Two

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:

seeds: 79 14 55 13

This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?


In [29]:
from typing import List, Dict, Union

def generate_additional_maps(seeds: List[int]) -> List[Dict[str, Union[str, List[Dict[str, Union[int, str]]]]]]:
    """
    Generate additional maps based on the seed values.

    Args:
    - seeds (List[int]): List of seed values.

    Returns:
    - List[Dict[str, Union[str, List[Dict[str, Union[int, str]]]]]]: List of dictionaries representing additional maps.

    Example:
    >>> generate_additional_maps([10, 20, 30])
    [{'source_type': 'start', 'destination_type': 'soil', 'map': [{'destination_range_start': 10, 'source_range_start': 10, 'range_length': 1, 'source_type': 'start', 'destination_type': 'soil'}, {'destination_range_start': 11, 'source_range_start': 10, 'range_length': 1, 'source_type': 'start', 'destination_type': 'soil'}, ...]}]
    """
    additional_maps = []

    for seed in seeds:
        # Create a map for each seed value
        seed_map = [{'destination_range_start': seed + i, 'source_range_start': seed, 'range_length': 1, 'source_type': 'start', 'destination_type': 'seed'} for i in range(seed, seed + 1)]
        
        additional_maps.append(seed_map[0])

    return {'source_type': 'start', 'destination_type': 'seed', 'map': additional_maps}

In [13]:
additional_maps = generate_additional_maps(seeds)

In [14]:
import copy

In [15]:
additional_processed_maps = copy.deepcopy(processed_maps)

In [16]:
additional_processed_maps = [additional_maps, *additional_processed_maps]

In [18]:
from typing import List

def is_seed_in_true_seeds(seed_list: List[int], target_seed: int) -> bool:
    """
    Check if an arbitrary seed number would be contained in the true seeds.

    Args:
    - seed_list (List[int]): The initial list of seeds with ranges.
    - target_seed (int): The seed number to check.

    Returns:
    - bool: True if the target seed is in the true seeds, False otherwise.

    Example:
    >>> is_seed_in_true_seeds([10, 3, 20, 5], 15)
    True
    >>> is_seed_in_true_seeds([10, 3, 20, 5], 25)
    False
    """
    for i in range(0, len(seed_list), 2):
        seed_value = seed_list[i]
        range_value = seed_list[i + 1]

        if seed_value <= target_seed < seed_value + range_value:
            return True

    return False

In [19]:
source_starts = set()
for map_type in additional_processed_maps:
    for map_detail in map_type['map']:
        for entry in map_detail:
            source_starts.add(map_detail['source_range_start'])
            source_starts.add(map_detail['source_range_start'] + map_detail['range_length'])
            source_starts.add(map_detail['source_range_start'] + map_detail['range_length'] + 1)
            source_starts.add(map_detail['destination_range_start'])
            source_starts.add(map_detail['destination_range_start'] + map_detail['range_length'])
            source_starts.add(map_detail['destination_range_start'] + map_detail['range_length'] + 1)
source_starts = [start for start in source_starts if is_seed_in_true_seeds(seeds, start)]

In [20]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.WARNING)

In [21]:
seed_locations = get_seed_locations(source_starts, additional_processed_maps, 'start', 'location')

In [22]:
import pandas as pd

In [24]:
seed_locations_df = pd.DataFrame(list(seed_locations.items()), columns=['seed', 'location'])

In [25]:
seed_locations_df['in_true_seeds'] = seed_locations_df['seed'].apply(lambda x: is_seed_in_true_seeds(seeds, x))

In [28]:
seed_locations_df[seed_locations_df['in_true_seeds'] == 1].sort_values('location').iloc[0]['location']

46294175

## Summary of part 2 from ChatGPT

Didn't really use ChatGPT as much for this part. It did not return a meaningful summary in this case